In [26]:
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, DateType
from pyspark.sql.functions import asc, desc, rank, col
from pyspark.sql.window import Window

In [27]:
from pyspark.sql import SparkSession
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'

spark_session = SparkSession.builder.appName("Lab2").getOrCreate()
sc = spark_session._sc
spark_session

In [28]:
# Создание схемы данных
progLangSchema = StructType([
    StructField("Language", StringType(), nullable=False),
    StructField("Url_language", StringType(), nullable=False)
])

# Чтение CSV файла с заданной схемой
dfProgLangs = spark_session.read.format("csv").schema(progLangSchema).load("programming-languages.csv")

# Вывод первых строк DataFrame
dfProgLangs.show(5)

+----------+--------------------+
|  Language|        Url_language|
+----------+--------------------+
|      name|       wikipedia_url|
|   A# .NET|https://en.wikipe...|
|A# (Axiom)|https://en.wikipe...|
|A-0 System|https://en.wikipe...|
|        A+|https://en.wikipe...|
+----------+--------------------+
only showing top 5 rows



In [29]:
# Получение первой строки DataFrame
firstRow = dfProgLangs.first()

# Преобразование DataFrame в RDD и обработка данных
progLangsList = dfProgLangs.rdd \
    .filter(lambda x: x != firstRow) \
    .map(lambda x: x["Language"]) \
    .collect()

# Вывод первых 10 элементов списка
progLangsList[:10]

['A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET',
 'ABSYS']

In [30]:
# Чтение XML файла с использованием формата xml и указанием rowTag
dfPostsSample = spark_session.read.format("xml").options(rowTag="row").load("posts_sample.xml")

# Печать DataFrame
dfPostsSample.show()

# Печать первой строки DataFrame
print(dfPostsSample.first())

+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+--------------------+--------------------+----------+
|_AcceptedAnswerId|_AnswerCount|               _Body|_ClosedDate|_CommentCount| _CommunityOwnedDate|       _CreationDate|_FavoriteCount|    _Id|   _LastActivityDate|       _LastEditDate|_LastEditorDisplayName|_LastEditorUserId|_OwnerDisplayName|_OwnerUserId|_ParentId|_PostTypeId|_Score|               _Tags|              _Title|_ViewCount|
+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+--------------------+----------

In [31]:
# Функция для определения языка по тегам в строке
def DefineLanguage(row):
    languageTag = None  # Инициализируем переменную для хранения найденного языка
    # Перебираем список языков для поиска соответствующих тегов
    for language in progLangsList:
        # Если тег языка найден в строке (в верхнем регистре)
        if '<' + language.upper() + '>' in row._Tags.upper():
            languageTag = language  # Присваиваем язык
            break  # Прерываем цикл, так как язык найден
    # Если язык найден, возвращаем кортеж с нужными значениями
    if languageTag is not None:
        return (row._Id, languageTag, row._CreationDate.year, row._ViewCount)

# Функция для проверки, попадает ли дата в указанный диапазон
def IsDateInRange(row):
    leftBorder = datetime(year=2010, month=1, day=1)  # Начало диапазона (1 января 2010)
    rightBorder = datetime(year=2020, month=12, day=31)  # Конец диапазона (31 декабря 2020)
    # Проверяем, что дата создания записи лежит в пределах диапазона
    return row._CreationDate >= leftBorder and row._CreationDate <= rightBorder


In [32]:
# Фильтруем данные по наличию тегов и диапазону дат, затем применяем DefineLanguage для обработки каждой строки
'''
Применяем функцию DefineLanguage для извлечения информации о языке, году и количестве просмотров
Фильтруем None значения, которые могут появиться, если DefineLanguage не нашел язык
Группируем данные по году (row[2]) и языку (row[1])
Преобразуем данные в формат (группа, количество просмотров), где группа состоит из года и языка
Суммируем количество просмотров для каждой пары (год, язык)
Преобразуем данные обратно в формат (год, язык, количество просмотров)
Преобразуем RDD в DataFrame с соответствующими названиями столбцов
'''
topLanguagesPerYear = dfPostsSample.rdd.filter(lambda row: row._Tags is not None and IsDateInRange(row))\
    .map(DefineLanguage)\
    .filter(lambda row: row is not None)\
    .keyBy(lambda row: (row[2], row[1]))\
    .map(lambda row: ((row[0][0], row[0][1]), row[1][3]))\
    .reduceByKey(lambda a, b: a + b)\
    .map(lambda row: (row[0][0], row[0][1], row[1]))\
    .toDF(('Year', 'Language', 'Count'))

# Выводим результат
topLanguagesPerYear.show()


+----+-----------+-------+
|Year|   Language|  Count|
+----+-----------+-------+
|2010|       Java| 563211|
|2010|        Sed|    447|
|2010|          R|  11087|
|2011|Objective-C| 218762|
|2011|        PHP| 243646|
|2013|       Java|1035218|
|2013| PowerShell| 154758|
|2013|       Bash|  72161|
|2013|          C|  37918|
|2014| JavaScript| 505138|
|2014|         Go|   2409|
|2014|       Bash|   6593|
|2014|       Perl|   6920|
|2014|          C|  87281|
|2016|        PHP|  98009|
|2016|       Curl|   9386|
|2016|       Java| 168691|
|2016|       Bash|   7419|
|2019|       Java|   5942|
|2015|Objective-C|  11295|
+----+-----------+-------+
only showing top 20 rows



In [33]:
# Определяем окно для вычисления ранга: по году и с сортировкой по убыванию количества просмотров
window_spec = Window.partitionBy("Year").orderBy(topLanguagesPerYear["Count"].desc())

# Применяем функцию rank() для вычисления ранга в каждой группе (по году)
topLanguagesRanks = topLanguagesPerYear.withColumn("rank", rank().over(window_spec))

# Фильтруем только топ-10 языков по году (с учетом ранга) и удаляем столбец с рангом
result = topLanguagesRanks.filter(topLanguagesRanks["rank"] <= 10).drop(col("rank"))

# Выводим результат
result.show()


+----+-----------+-------+
|Year|   Language|  Count|
+----+-----------+-------+
|2010|        PHP|1184584|
|2010|       Java| 563211|
|2010| JavaScript| 316131|
|2010|Objective-C|  97009|
|2010|       Ruby|  76001|
|2010|          C|  66587|
|2010|     Python|  59392|
|2010|     MATLAB|  51865|
|2010|AppleScript|  32305|
|2010|     Delphi|  11817|
|2011| JavaScript| 806948|
|2011|       Java| 388524|
|2011|        PHP| 243646|
|2011|          C| 238277|
|2011|Objective-C| 218762|
|2011|     Python| 195016|
|2011|       Bash|  60805|
|2011|       Ruby|  33037|
|2011|       Perl|  24465|
|2011|     MATLAB|  18816|
+----+-----------+-------+
only showing top 20 rows

